In [29]:
import os
import json
import numpy as np
import pandas as pd

In [30]:
movies_df = pd.read_csv("./original-dataset/movies.csv")

In [31]:
with open('movie_to_relevant_tags.json', 'r') as fp:
        movie_to_tag_relevance = json.load(fp)

In [46]:
def createNode(id, varPrefix, type, attrs):
    return f"({varPrefix}{id}:{type} {json.dumps(attrs)}) return {varPrefix}{id}.id;\n"

print(createNode(1, "m", "Movie", {"id": 1, "name": "movie1"}))

def createEdge(id1, id2, varPrefix1, varPrefix2, type1, type2, relationshipType, attrs):
    var1 = f"{varPrefix1}{id1}"
    var2 = f"{varPrefix2}{id2}"
    return f"match ({var1}:{type1}), ({var2}:{type2}) where {var1}.id={id1} and {var2}.id={id2} merge ({var1})-[r:{relationshipType} {json.dumps(attrs)}]-({var2});\n"

print(createEdge(1, 1, "m", "g", "Movie", "Genre", "InGenre", {"name": "is genre of"}))

(m1:Movie {"id": 1, "name": "movie1"}) return m1.id;

match (m1:Movie), (g1:Genre) where m1.id=1 and g1.id=1 merge (m1)-[r:InGenre {"name": "is genre of"}]-(g1);



In [47]:
# creating nodes
cypherScript = ""
genreIndex = 1
tagIndex = 1
allGenres = set([])
allTags = set([])

for y, row in movies_df.iterrows():
    cypherScript += createNode(row["movieId"], "m", "Movie", {"id": row["movieId"], "title": row["title"]})

    genres = row["genres"].split("|")

    for i, genre in enumerate(genres):
        if not genre in allGenres:
            cypherScript += createNode(genreIndex+i, "g", "Genre", {"id": genreIndex+i, "name": genre})
            allGenres.add(genre)

    genreIndex += len(genres)

    print(f"{y}/{len(movies_df)}", end="\r")

print("\n")

for y, movieId in enumerate(movie_to_tag_relevance):
    tags = movie_to_tag_relevance[movieId]

    for i, tag in enumerate(tags):
        if not tag in allTags:
            cypherScript += createNode(tagIndex+i, "t", "Tag", {"id": tagIndex+i, "name": tag})

    tagIndex += len(tags)

    print(f"{y}/{len(movie_to_tag_relevance)}", end="\r")

27277/27278



In [48]:
print(allGenres)

{'Sci-Fi', 'Documentary', 'Fantasy', 'Musical', 'Animation', 'Children', 'Romance', 'Thriller', 'War', 'IMAX', 'Horror', 'Drama', 'Adventure', 'Film-Noir', 'Comedy', 'Mystery', 'Western', 'Action', '(no genres listed)', 'Crime'}


In [49]:
with open("create_nodes.txt", "w") as f:
    f.write(cypherScript)